In [22]:
#! pip install seqeval
#! pip install evaluate
#! pip install pandas
#! pip install datasets
#! pip install torch
#! pip install transformers
#! pip install scikit-learn
#! pip install ninja
#! pip install flash-attn
#! pip install packaging
#! pip install flash-attn --no-build-isolation

  Using cached flash_attn-2.7.4.post1.tar.gz (6.0 MB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\magnu\\AppData\\Local\\Temp\\pip-install-_h0lj359\\flash-attn_a64e364bab4b4bda827435c2541f476a\\csrc\\composable_kernel\\library\\include\\ck\\library\\tensor_operation_instance\\gpu\\grouped_conv_bwd_weight\\device_grouped_conv_bwd_weight_two_stage_xdl_instance.hpp'


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [84]:
import json
import torch
import evaluate
import seqeval
import accelerate
import transformers
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from sklearn.model_selection import train_test_split

#from flash_attn.flash_attention import FlashAttention
# Flash Attention for faster training

torch.backends.cuda.matmul.allow_tf32 = True  # Enable TF32 for better performance
torch.backends.cudnn.benchmark = True  # Enable CuDNN auto-tuner

In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Load dataset
DATA_PATH = "tokenized_ner_data_6.json"
with open(DATA_PATH, "r") as f:
    data = json.load(f)

In [4]:
dataset = Dataset.from_list(data)
testing_dataset = Dataset.from_list(data)
train_dataset, eval_dataset = dataset.train_test_split(test_size=0.2).values()

In [91]:
print(dataset)
#print(train_dataset)
#print(eval_dataset)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'attention_mask', 'word_ids'],
        num_rows: 48
    })
    test: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'attention_mask', 'word_ids'],
        num_rows: 13
    })
})


In [6]:
unique_labels = set(label for entry in dataset for label in entry["labels"])
label2id = {label: i for i, label in enumerate(sorted(unique_labels))}
id2label = {i: label for label, i in label2id.items()}

print(label2id)  # Mapping of labels to IDs

# Convert text labels to integer IDs
#data["labels"] = [label2id[label] for label in data["labels"]]

{'B-CVE': 0, 'B-Event': 1, 'B-LOC': 2, 'B-MAL-ORG': 3, 'B-MISC': 4, 'B-Malware': 5, 'B-ORG': 6, 'B-PER': 7, 'B-Software': 8, 'I-CVE': 9, 'I-Event': 10, 'I-LOC': 11, 'I-MAL-ORG': 12, 'I-MISC': 13, 'I-Malware': 14, 'I-ORG': 15, 'I-PER': 16, 'I-Software': 17, 'O': 18}


In [7]:
# Load tokenizer and model
MODEL_NAME = "google-bert/bert-base-cased"
#"answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Enable Flash Attention in the model (if applicable)
#model.config.use_flash_attention = True

In [8]:
def process_tokens_and_labels(data):
    """Convert tokenized text into IDs and align labels."""
    data["input_ids"] = tokenizer.convert_tokens_to_ids(data["tokens"])
    data["attention_mask"] = [1] * len(data["input_ids"])  # Mask for all tokens
    
    # Infer word_ids manually
    word_ids = []
    word_idx = -1 # Initialize
    for token in data["tokens"]:
        if token in ["[CLS]", "[SEP]"]:
            word_ids.append(None)
        elif token.startswith("##"):
            word_ids.append(word_idx)
        else:
            word_idx += 1
            word_ids.append(word_idx)

    data["word_ids"] = word_ids
    
    # Convert text labels to integer IDs
    data["labels"] = [label2id[label] for label in data["labels"]]

    return data

# Apply transformation to dataset
dataset = dataset.map(process_tokens_and_labels)

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

In [9]:
print(dataset[0])

{'tokens': ['[CLS]', 'A', 'new', 'ransom', '##ware', '-', 'as', '-', 'a', '-', 'service', '(', 'Ra', '##a', '##S', ')', 'operation', 'named', 'C', '##ica', '##da', '##33', '##01', 'has', 'already', 'listed', '19', 'victims', 'on', 'its', 'ex', '##tor', '##tion', 'portal', ',', 'as', 'it', 'quickly', 'attacked', 'companies', 'worldwide', '.', 'The', 'new', 'c', '##y', '##ber', '##c', '##rim', '##e', 'operation', 'is', 'named', 'after', 'the', 'mysterious', '2012', '-', '2014', 'online', '/', 'real', '-', 'world', 'game', 'that', 'involved', 'elaborate', 'cry', '##pt', '##ographic', 'puzzles', 'and', 'used', 'the', 'same', 'logo', 'for', 'promotion', 'on', 'c', '##y', '##ber', '##c', '##rim', '##e', 'forums', '.', 'However', ',', 'there', "'", 's', 'no', 'connection', 'between', 'the', 'two', ',', 'and', 'the', 'legitimate', 'project', 'has', 'issued', 'a', 'statement', 'to', 're', '##nounce', 'any', 'association', 'and', 'con', '##de', '##m', '##n', 'the', 'ransom', '##ware', 'operators

In [72]:
for i, (label, word_id) in enumerate(zip(dataset[0]['labels'], dataset[0]['word_ids'])):
    print(f"Index {i}: Label {label}, Word ID {word_id}")

Index 0: Label 18, Word ID None
Index 1: Label 18, Word ID 0
Index 2: Label 18, Word ID 1
Index 3: Label 18, Word ID 2
Index 4: Label 18, Word ID 2
Index 5: Label 18, Word ID 3
Index 6: Label 18, Word ID 4
Index 7: Label 18, Word ID 5
Index 8: Label 18, Word ID 6
Index 9: Label 18, Word ID 7
Index 10: Label 18, Word ID 8
Index 11: Label 18, Word ID 9
Index 12: Label 18, Word ID 10
Index 13: Label 18, Word ID 10
Index 14: Label 18, Word ID 10
Index 15: Label 18, Word ID 11
Index 16: Label 18, Word ID 12
Index 17: Label 18, Word ID 13
Index 18: Label 3, Word ID 14
Index 19: Label 12, Word ID 14
Index 20: Label 12, Word ID 14
Index 21: Label 12, Word ID 14
Index 22: Label 12, Word ID 14
Index 23: Label 18, Word ID 15
Index 24: Label 18, Word ID 16
Index 25: Label 18, Word ID 17
Index 26: Label 18, Word ID 18
Index 27: Label 18, Word ID 19
Index 28: Label 18, Word ID 20
Index 29: Label 18, Word ID 21
Index 30: Label 18, Word ID 22
Index 31: Label 18, Word ID 22
Index 32: Label 18, Word ID 

In [35]:
for i in range(30):
    print("Token: ", dataset[0]['tokens'][i])
    print("Label: ", dataset[0]['labels'][i])
    print("Word_ids: ", dataset[0]['word_ids'][i])
    print("Input_ids: ", dataset[0]['input_ids'][i])
    print("---------------------------")

Token:  [CLS]
Label:  18
Word_ids:  None
Input_ids:  101
---------------------------
Token:  A
Label:  18
Word_ids:  0
Input_ids:  138
---------------------------
Token:  new
Label:  18
Word_ids:  1
Input_ids:  1207
---------------------------
Token:  ransom
Label:  18
Word_ids:  2
Input_ids:  25057
---------------------------
Token:  ##ware
Label:  18
Word_ids:  2
Input_ids:  7109
---------------------------
Token:  -
Label:  18
Word_ids:  3
Input_ids:  118
---------------------------
Token:  as
Label:  18
Word_ids:  4
Input_ids:  1112
---------------------------
Token:  -
Label:  18
Word_ids:  5
Input_ids:  118
---------------------------
Token:  a
Label:  18
Word_ids:  6
Input_ids:  170
---------------------------
Token:  -
Label:  18
Word_ids:  7
Input_ids:  118
---------------------------
Token:  service
Label:  18
Word_ids:  8
Input_ids:  1555
---------------------------
Token:  (
Label:  18
Word_ids:  9
Input_ids:  113
---------------------------
Token:  Ra
Label:  18
Word_ids: 

In [73]:
def align_labels_2(data):
    word_ids = data["word_ids"]
    original_labels = data["labels"]
    new_labels = []

    previous_word_idx = None
    
    for idx, (word_id, label) in enumerate(zip(word_ids, original_labels)):
        if word_id is None:
            new_labels.append(-100) # Special tokens
        elif word_id != previous_word_idx:
            new_labels.append(label)
        else:
            new_labels.append(-100)
        previous_word_idx = word_id

    data["labels"] = new_labels
    return data

In [74]:
tokenized_and_aligned_dataset = dataset.map(align_labels_2)

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

In [75]:
print(tokenized_and_aligned_dataset[0])

{'tokens': ['[CLS]', 'A', 'new', 'ransom', '##ware', '-', 'as', '-', 'a', '-', 'service', '(', 'Ra', '##a', '##S', ')', 'operation', 'named', 'C', '##ica', '##da', '##33', '##01', 'has', 'already', 'listed', '19', 'victims', 'on', 'its', 'ex', '##tor', '##tion', 'portal', ',', 'as', 'it', 'quickly', 'attacked', 'companies', 'worldwide', '.', 'The', 'new', 'c', '##y', '##ber', '##c', '##rim', '##e', 'operation', 'is', 'named', 'after', 'the', 'mysterious', '2012', '-', '2014', 'online', '/', 'real', '-', 'world', 'game', 'that', 'involved', 'elaborate', 'cry', '##pt', '##ographic', 'puzzles', 'and', 'used', 'the', 'same', 'logo', 'for', 'promotion', 'on', 'c', '##y', '##ber', '##c', '##rim', '##e', 'forums', '.', 'However', ',', 'there', "'", 's', 'no', 'connection', 'between', 'the', 'two', ',', 'and', 'the', 'legitimate', 'project', 'has', 'issued', 'a', 'statement', 'to', 're', '##nounce', 'any', 'association', 'and', 'con', '##de', '##m', '##n', 'the', 'ransom', '##ware', 'operators

In [76]:
for i in range(30):
    print("Token: ", tokenized_and_aligned_dataset[0]['tokens'][i])
    print("Label: ", tokenized_and_aligned_dataset[0]['labels'][i])
    print("Word_ids: ", tokenized_and_aligned_dataset[0]['word_ids'][i])
    print("Input_ids: ", tokenized_and_aligned_dataset[0]['input_ids'][i])
    print("---------------------------")

Token:  [CLS]
Label:  -100
Word_ids:  None
Input_ids:  101
---------------------------
Token:  A
Label:  18
Word_ids:  0
Input_ids:  138
---------------------------
Token:  new
Label:  18
Word_ids:  1
Input_ids:  1207
---------------------------
Token:  ransom
Label:  18
Word_ids:  2
Input_ids:  25057
---------------------------
Token:  ##ware
Label:  -100
Word_ids:  2
Input_ids:  7109
---------------------------
Token:  -
Label:  18
Word_ids:  3
Input_ids:  118
---------------------------
Token:  as
Label:  18
Word_ids:  4
Input_ids:  1112
---------------------------
Token:  -
Label:  18
Word_ids:  5
Input_ids:  118
---------------------------
Token:  a
Label:  18
Word_ids:  6
Input_ids:  170
---------------------------
Token:  -
Label:  18
Word_ids:  7
Input_ids:  118
---------------------------
Token:  service
Label:  18
Word_ids:  8
Input_ids:  1555
---------------------------
Token:  (
Label:  18
Word_ids:  9
Input_ids:  113
---------------------------
Token:  Ra
Label:  18
Word_i

In [79]:
dataset = dataset.map(align_labels_2)

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

In [80]:
for i in range(30):
    print("Token: ", dataset[0]['tokens'][i])
    print("Label: ", dataset[0]['labels'][i])
    print("Word_ids: ", dataset[0]['word_ids'][i])
    print("Input_ids: ", dataset[0]['input_ids'][i])
    print("---------------------------")

Token:  [CLS]
Label:  -100
Word_ids:  None
Input_ids:  101
---------------------------
Token:  A
Label:  18
Word_ids:  0
Input_ids:  138
---------------------------
Token:  new
Label:  18
Word_ids:  1
Input_ids:  1207
---------------------------
Token:  ransom
Label:  18
Word_ids:  2
Input_ids:  25057
---------------------------
Token:  ##ware
Label:  -100
Word_ids:  2
Input_ids:  7109
---------------------------
Token:  -
Label:  18
Word_ids:  3
Input_ids:  118
---------------------------
Token:  as
Label:  18
Word_ids:  4
Input_ids:  1112
---------------------------
Token:  -
Label:  18
Word_ids:  5
Input_ids:  118
---------------------------
Token:  a
Label:  18
Word_ids:  6
Input_ids:  170
---------------------------
Token:  -
Label:  18
Word_ids:  7
Input_ids:  118
---------------------------
Token:  service
Label:  18
Word_ids:  8
Input_ids:  1555
---------------------------
Token:  (
Label:  18
Word_ids:  9
Input_ids:  113
---------------------------
Token:  Ra
Label:  18
Word_i

In [96]:
# Verify split
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'attention_mask', 'word_ids'],
        num_rows: 48
    })
    validation: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'attention_mask', 'word_ids'],
        num_rows: 13
    })
})


In [97]:
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

C:\Users\magnu\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\magnu\.cache\huggingface\hub\models--google-bert--bert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BertForTokenClassification were not initialized from the model checkpo

In [98]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [99]:
# Load metric
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_labels = [[label for label in label_seq if label != -100] for label_seq in labels]
    true_predictions = [[pred for pred, lab in zip(pred_seq, label_seq) if lab != -100] for pred_seq, label_seq in zip(predictions, labels)]
    return seqeval.compute(predictions=true_predictions, references=true_labels)

In [103]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2
)

In [104]:
# Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)


C:\Users\magnu\AppData\Local\Temp\ipykernel_27368\3993666119.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [105]:
# Train the model
trainer.train()

# Save the model
trainer.save_model("./ner_model")
tokenizer.save_pretrained("./ner_model")

Epoch,Training Loss,Validation Loss
1,No log,0.807386
2,1.254900,0.696961
3,1.254900,0.587973
4,0.600000,0.549310
5,0.492200,0.527802


('./ner_model\\tokenizer_config.json',
 './ner_model\\special_tokens_map.json',
 './ner_model\\vocab.txt',
 './ner_model\\added_tokens.json',
 './ner_model\\tokenizer.json')